# Change Catalog Object Owners
This notebook uses [DiscoverX](https://github.com/databrickslabs/discoverx) to change the owner of every object in a catalog.

Provide the catalog and the new owner below, then run all cells.

In [ ]:
%pip install dbl-discoverx
dbutils.library.restartPython()


## Configure Catalog and New Owner

In [ ]:

# Widgets for user input
try:
    dbutils.widgets.text("1.catalog", "catalog")
    dbutils.widgets.text("2.new_owner", "account users")
    catalog = dbutils.widgets.get("1.catalog")
    new_owner = dbutils.widgets.get("2.new_owner")
except NameError:
    # When running as a Python script
    catalog = "source_catalog"
    new_owner = "account users"


## Change Schema Owners

In [ ]:
schemas_df = spark.sql(f"SHOW SCHEMAS IN `{catalog}`")
for row in schemas_df.collect():
    schema = row[0]
    if schema.lower() in ("information_schema",):
        continue
    print(f"Changing owner of object `{catalog}`.`{schema}` to `{new_owner}`")
    spark.sql(f"ALTER SCHEMA `{catalog}`.`{schema}` OWNER TO `{new_owner}`")

## Change Table Owners using DiscoverX

In [ ]:
from discoverx import DX

dx = DX()

def set_table_owner(tbl_info):
    if tbl_info.schema.lower() in ("information_schema",):
        return {
            "table": f"`{tbl_info.catalog}`.`{tbl_info.schema}`.`{tbl_info.table}`",
            "success": True,
            "info": "skipped system schema",
        }
    try:
        print(f"Changing owner of object `{tbl_info.catalog}`.`{tbl_info.schema}`.`{tbl_info.table}` to `{new_owner}`")
        spark.sql(
            f"ALTER TABLE `{tbl_info.catalog}`.`{tbl_info.schema}`.`{tbl_info.table}` OWNER TO `{new_owner}`"
        )
        return {
            "table": f"`{tbl_info.catalog}`.`{tbl_info.schema}`.`{tbl_info.table}`",
            "success": True,
            "info": None,
        }
    except Exception as err:
        return {
            "table": f"`{tbl_info.catalog}`.`{tbl_info.schema}`.`{tbl_info.table}`",
            "success": False,
            "info": str(err),
        }

results = dx.from_tables(f"{catalog}.*.*").map(set_table_owner)

## Change Volume Owners

In [ ]:
volume_df = spark.sql(f"""
SELECT volume_schema, volume_name
FROM system.information_schema.volumes
WHERE volume_catalog = '{catalog}'
""")
for row in volume_df.collect():
    if row.volume_schema.lower() in ("information_schema",):
        continue
    print(f"Changing owner of object `{catalog}`.`{row.volume_schema}`.`{row.volume_name}` to `{new_owner}`")
    spark.sql(
        f"ALTER VOLUME `{catalog}`.`{row.volume_schema}`.`{row.volume_name}` OWNER TO `{new_owner}`"
    )

## Change Function Owners

In [ ]:
func_df = spark.sql(f"""
SELECT routine_schema, routine_name
FROM system.information_schema.routines
WHERE routine_catalog = '{catalog}'
""")
for row in func_df.collect():
    if row.routine_schema.lower() in ("information_schema",):
        continue
    print(f"Changing owner of object `{catalog}`.`{row.routine_schema}`.`{row.routine_name}` to `{new_owner}`")
    spark.sql(
        f"ALTER FUNCTION `{catalog}`.`{row.routine_schema}`.`{row.routine_name}` OWNER TO `{new_owner}`"
    )

## Results

In [ ]:
import json
for item in results:
    print(json.dumps(item, indent=4))